In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os
from UQpy.RunModel import RunModel

In [2]:
# Parameters of the STZ model

#BETA : (2 - 18eV)
#U0 :   (np.amin(pe.MD.0) - 0.01)  —  (np.amax(pe.MD.0) - 0.001)
#CHI_LEN :10 -100 Angstrom
#KAPPA :10 -100 Angstrom
#C0 :0.1 - 0.9
#EPS0 :0.1 - 0.9
#OMEGA :60 - 300 Cubic Angstroms
#THETA :100K
#DELTA :500 - 10000K
#CHI_INF : BETA * (np.amax(pe.MD.100) - U0 ) * TZ, TZ = 21000
#SY :0.85Gpa
#MU :20GPa
#K :143.23GPa
#RHO :7234

In [3]:
peMD0 = np.fromfile('pe.MD.0', dtype=np.float32)
peMD100 = np.fromfile('pe.MD.100', dtype=np.float32)
U0min = (np.amin(peMD0) - 0.01)
U0max = (np.amax(peMD0) - 0.001)
print(U0min, U0max)

-0.01 -0.0008233570950105787


In [4]:
TZ = 21000
beta = 4.7958
u0 = -3.3625
#chi_len =15.05
chi_len = 10 #use 2 (increase this reduce the resolution)
kappa = 0.06
c0 = 0.414
eps0 = 0.333
Omega =349
theta = 97
Delta = 7948

Uf_MD = np.loadtxt('pe.MD.100', skiprows=1)
Tf_MD = beta*(Uf_MD-u0)*TZ
Chi_inf = np.amax(Tf_MD)
    
sy = 0.7
mu = 20
K = 143.23
rho = 7234

RunModelObject = RunModel(model_script='pfn.py',model_object_name="STZmodel")
samples = np.array([beta, u0, chi_len, kappa, c0, eps0, Omega, theta, Delta, Chi_inf, sy, mu, K, rho])

In [5]:
RunModelObject.run(samples.reshape(1, -1))

FileNotFoundError: [Errno 2] No such file or directory: './sct_q.out/Exy.100'

In [ ]:
coordinates = np.loadtxt('coordinates.txt')
coordinates = coordinates[1:]
X, Y = np.meshgrid(coordinates, coordinates)

In [ ]:
# Interpolate
import scipy.interpolate
Z = RunModelObject.qoi_list[0]
rbf = scipy.interpolate.Rbf(X, Y, Z, function='cubic')
zi = rbf(X, Y)

plt.imshow(zi, vmin=Z.min(), vmax=Z.max(), origin='lower',
           extent=[X.min(), X.max(), Y.min(), Y.max()])
plt.scatter(X, Y, c=Z)
plt.colorbar()
plt.show()